## 2 (Remark: I used R instead of STATA)

## Generate data

In [403]:
library(mvtnorm)
n =1000
set.seed(10)
mu = c(0, 0)
 A <- array(c(0.5, 0.25, 0.25, 4 ), dim=c(2,2))
w = rmvnorm(n, mu, A)
u = w[,1]
v = w[,2]
# plot(w)
x = rnorm(n, mean = 0 ,sd = 1)
z = rnorm(n, mean = 0 ,sd = 1)

In [404]:
df    <- data.frame(u=u, v=v, x=x , z =z) 
# head(df)

In [405]:
D = sign(1 + z + v)
Ys = 1 + x + u
df    <- data.frame(df ,dc =1 + z + v, D =D, Ys = Ys)

In [406]:
#replace the result of sign func.
df$D[df$D == 0] = 1 
df$D[df$D == -1] = 0 

In [407]:
head(df)

,u,v,x,z,dc,D,Ys
1,-0.003926272,-0.3663731,-0.307865,0.4605151,1.094142,1,0.6882087
2,-1.016823,-1.324078,0.7580856,0.2350253,-0.08905249,0,0.7412631
3,0.2425872,0.8060361,-0.5738634,0.6432573,2.449293,1,0.6687238
4,-0.8805651,-0.8384773,-0.9387445,0.9131981,1.074721,1,-0.8193096
5,-1.164077,-0.6630874,-0.02769932,0.988286,1.325199,1,-0.1917762
6,0.8423711,1.612,-1.066249,0.1127413,2.724742,1,0.7761224


## (b) Heckit by hand  
 
 

### 1. Probit estimatin

In [408]:
res <- glm(D ~  z , data = df, family=binomial(probit))
summary(res)
    res$coefficients[2]


Call:
glm(formula = D ~ z, family = binomial(probit), data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6420  -1.1581   0.6316   0.8885   1.5408  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.51812    0.04374   11.85   <2e-16 ***
z            0.46218    0.04549   10.16   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1256.7  on 999  degrees of freedom
Residual deviance: 1142.2  on 998  degrees of freedom
AIC: 1146.2

Number of Fisher Scoring iterations: 4


z 
0.4621761

### 2. Inverse Mill's Ratio

In [409]:
lbd = dnorm(z*res$coefficients[2])/pnorm(z*res$coefficients[2])
head(lbd)
df    <- data.frame(df ,lbd= lbd)

[1] 0.6675085 0.7300436 0.6187513 0.5500248 0.5316292 0.7650113

### 3. Regression

In [410]:
res <- lm(Ys ~  x + lbd, data = df)
summary(res)


Call:
lm(formula = Ys ~ x + lbd, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.32577 -0.45767  0.01295  0.49208  2.51733 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.06187    0.06723   15.79   <2e-16 ***
x            1.02585    0.02249   45.62   <2e-16 ***
lbd         -0.06732    0.07654   -0.88    0.379    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7228 on 997 degrees of freedom
Multiple R-squared:  0.6766,	Adjusted R-squared:  0.6759 
F-statistic:  1043 on 2 and 997 DF,  p-value: < 2.2e-16


## (c) 2-step estimation

In [411]:
library("sampleSelection")

In [412]:
a <- heckit( D ~  z, Ys ~ x , data=df,method = "2step")

In [413]:
coef(summary(a))

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),5.181239e-01,4.371983e-02,1.185101e+01,2.123958e-30
z,4.621762e-01,4.561822e-02,1.013140e+01,5.029723e-23
(Intercept),1.071536e+00,6.688249e-02,1.602117e+01,1.462510e-51
x,1.012247e+00,2.712527e-02,3.731750e+01,2.915670e-191
invMillsRatio,-0.009687247,0.127574552,-0.075934009,0.939486879
sigma,0.7163367,NA,NA,NA
rho,-0.01352332,NA,NA,NA


**Std. error of (b): 0.02250  ・・・(1)  
Std. error of (c): 0.02712  ・・・(2)**
  
Here we can see that two std. errors are different.  In this case (2) > (1) , however it is (1) > (2) in STATA as I saw in some of my friends result.  It is because  when we calculate standard error we must correct an asymptoticvariance.  In this case , heckit, we should replace 2-stage estimatour with GMM estimatour.  STATA does thi automatically, thus (1) > (2) happens in STATA.

 Remark: I have no idea why the opposite happened in R. Perhaps it is because glm() command in R uses more appropriate way to in its inside.

## (a) Convergence in Probability

In [421]:

test <- function(n) {
set.seed(10)

mu = c(0, 0)
A <- array(c(0.5, 0.25, 0.25, 4 ), dim=c(2,2))
w = rmvnorm(n, mu, A)
u = w[,1]
v = w[,2]
x = rnorm(n, mean = 0 ,sd = 1)
z = rnorm(n, mean = 0 ,sd = 1)
df    <- data.frame(u=u, v=v, x=x , z =z) 
D = sign(1 + z + v)
Ys = 1 + x + u
df    <- data.frame(df ,dc =1 + z + v, D =D, Ys = Ys)
df$D[df$D == 0] = 1 
df$D[df$D == -1] = 0 
res <- glm(D ~  z , data = df, family=binomial(probit))
return(res)

}

print("N = 1")
test(1)$coefficients[2]
print("N = 10")
test(10)$coefficients[2]
print("N = 100")
test(100)$coefficients[2]
print("N = 10000")
test(1000)$coefficients[2]
print("N = 100000")
test(10000)$coefficients[2]
print("N = 1000000")
test(100000)$coefficients[2]
print("N = 10000000")
test(1000000)$coefficients[2]

[1] "N = 1"


z 
NA

[1] "N = 10"


z 
1.046096

[1] "N = 100"


...

z 
0.502342

[1] "N = 1000000"


z 
0.4996398

[1] "N = 10000000"


z 
0.5023313

As shown in above, the coefficient converges to about 0.5.